In [1]:

import spacy
import pandas as pd
import json
import os
import re

from spacy.pipeline import EntityRuler
from spacy.lang.en import English

#nlp=English()
#arxikopoihsh tou nlp kai entity ruler
nlp=spacy.blank("en")
ruler=nlp.add_pipe("entity_ruler")



In [16]:
#in order to convert ubiai IOB format into spacy input for training
#gia na to diavasei swsta esvhsa thn prwth grammh kai evala no header
import re
import pandas as pd


train_all = pd.read_csv("training_IOBall.tsv", sep='\t+', header=None, engine='python') #skiprows=1,
dev_all = pd.read_csv("development_IOBall.tsv", sep='\t+', header=None, engine='python')
eval_all = pd.read_csv("evaluation_IOBall.tsv", sep='\t+', header=None, engine='python')

# print(train_all)
def iob_to_spacy(df):
    words = df[0]
    tags = df[1]
    total_words = len(words)
    data = []
    data_i = 0
    i = 0

    sentence = ""
    sentence_len = 0
    sentence_entities = []
    for word in words:
        dot_index = word.find('.')
        if (dot_index != -1):
            # vrethike sth thesh dot_index
            if (dot_index == len(word)-1):
                # einai telefteo gramma
                # prepei na checkarw to 1o gramma sto epomeno word
                if (total_words-1 > i):
                    # uparxei to epomeno word
                    # elegxw to 1o gramma an einai kefaleo
                    if (words[i+1][0].isupper()):
                        sentence += word
                        if (tags[i] != 'O'):
                            entity = (sentence_len, len(sentence), tags[i])
                            sentence_entities.append(entity)
                        sentence_len = len(sentence)
                        new_val = (sentence, {"entities": sentence_entities})
                        data.append(new_val)
                        sentence = ""
                        sentence_len = 0
                        sentence_entities = []
                    else:
                        sentence += word + " "
                        if (tags[i] != 'O'):
                            entity = (sentence_len, len(sentence), tags[i])
                            sentence_entities.append(entity)
                        sentence_len = len(sentence)
        else:
            sentence += word + " "
            if (tags[i] != 'O'):
                entity = (sentence_len, len(sentence), tags[i])
                sentence_entities.append(entity)
            sentence_len = len(sentence)
        i += 1
    for sent in data:
        entities = sent[1]['entities']
        new_entities = []
        entity_start = 0
        entity_end = 0
        entity_type = ""
        for entity in entities:
            if entity[2][0] == "B":
                if entity_end != 0:
                    new_entity = (entity_start, entity_end, entity_type)
                    new_entities.append(new_entity)
                entity_start = entity[0]
                entity_end = entity[1]
                entity_type = entity[2]
            else:
                entity_end = entity[1]
        if entity_end != 0:
            new_entity = (entity_start, entity_end, entity_type)
            new_entities.append(new_entity)
            sent[1]['entities']=new_entities

    for sent in data:
        entity_start = 0
        entity_end = 0
        entity_type = ""
        new_entities=[]
        for entity in sent[1]['entities']:
            entity_start = entity[0]
            entity_end = entity[1]
            entity_type = entity[2]
            entity_type=re.sub(r'.', '', entity_type , count = 2)
            new_entity = (entity_start, entity_end, entity_type)
            new_entities.append(new_entity)
            sent[1]['entities']=new_entities
    return data

train_fulltext=iob_to_spacy(train_all)
dev_fulltext=iob_to_spacy(dev_all)
eval_fulltext=iob_to_spacy(eval_all)


# print(train_fulltext)

In [40]:
df=pd.read_csv("dictionaries-annot\Distribution_descriptors.csv",header=None)
df1=pd.read_csv("dictionaries-annot\Life_stages.csv",header=None)
df2=pd.read_csv("dictionaries-annot\Body_size.csv",header=None)
df3=pd.read_csv("dictionaries-annot\Sampling_devices.csv",header=None,encoding='cp1252')

df_distr_descr=df.iloc[:,0]
df_life_stages=df1.iloc[:,0]
df_body_size=df2.iloc[:,0]
df_biol_descr=df3.iloc[:,0]



In [5]:
import random
import spacy.cli
from spacy.tokens import DocBin
from tqdm import tqdm

In [17]:
#in order to create training data,custom ner model

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe('ner')

# ner.add_label("Distribution_descriptor")
# ner.add_label("Life_stage")
# ner.add_label("Body_size")
# ner.add_label("Sampling_device")
ner.add_label("DISTRIBUTION_DESCRIPTOR ")
ner.add_label("LIFE_STAGE")
ner.add_label("BODY_SIZE")
ner.add_label("SAMPLING_DEVICE")
ner.labels


('BODY_SIZE', 'DISTRIBUTION_DESCRIPTOR ', 'LIFE_STAGE', 'SAMPLING_DEVICE')

In [18]:
from spacy.util import filter_spans

def training_data_format(train_data,name):
    doc_bin = DocBin()
    for training_example in tqdm(train_data): 
        text = training_example[0]
        # print(text)
        labels = training_example[1]['entities']
        # print(training_example[1])
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in labels:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents 
        doc_bin.add(doc)
    
    return(doc_bin.to_disk("train_" + name + ".spacy"))


training_data_format(train_fulltext,"train_full")
training_data_format(dev_fulltext,"dev_full")
training_data_format(eval_fulltext,"eval_full")


100%|██████████| 707/707 [00:00<00:00, 7364.55it/s]


In [19]:
#configuration file initiallization
!python -m spacy init fill-config base_config.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [20]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' #for error fixing
!python -m spacy train config.cfg --output ./ --paths.train ./train_train_full.spacy --paths.dev ./train_eval_full.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     35.00    0.00    0.00    0.00    0.00
  1     200         36.79   1567.91   26.62   24.18   29.60    0.27
  2     400         68.53   1115.46   30.84   34.31   28.00    0.31
  4     600         53.19   1155.08   45.43   35.98   61.60    0.45
  6     800        229.58   1081.01   34.62   33.33   36.00    0.35
  9    1000        100.33    863.77   21.83   15.02   40.00    0.22
 12    1200        233.04    804.37   37.66   39.47   36.00    0.38
 16    1400        183.36    720.22   35.63   36.07   35.20    0.36
 21    1600        178.14    606.85 

[2023-01-30 10:07:05,156] [INFO] Set up nlp object from config
[2023-01-30 10:07:05,166] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-01-30 10:07:05,170] [INFO] Created vocabulary
[2023-01-30 10:07:06,793] [INFO] Added vectors: en_core_web_lg
[2023-01-30 10:07:08,121] [INFO] Finished initializing nlp object
[2023-01-30 10:07:08,813] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [38]:
from spacy import displacy

nlp_full=spacy.load("model-best_fulldev-eval")
doc=nlp_full("Atherinids are small marine, estuarine and freshwater fishes not exceeding 120 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis (predominantly in the shore waters of the Caribbean) known from the New World. I")

print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in doc.ents])
print("\n")

displacy.render(doc, style="ent")


[('marine', 'DISTRIBUTION_DESCRIPTOR', 21, 27, ''), ('estuarine', 'DISTRIBUTION_DESCRIPTOR', 29, 38, ''), ('freshwater', 'DISTRIBUTION_DESCRIPTOR', 43, 53, ''), ('120 mm SL', 'BODY_SIZE', 75, 84, ''), ('300 mm SL', 'BODY_SIZE', 146, 155, ''), ('Alepidomus evermanni', 'DISTRIBUTION_DESCRIPTOR', 210, 230, ''), ('marine', 'DISTRIBUTION_DESCRIPTOR', 262, 268, ''), ('shore waters', 'DISTRIBUTION_DESCRIPTOR', 355, 367, '')]




In [ ]:
from spacy.tokenizer import Tokenizer
tokenizer = nlp.tokenizer

#in order to put the entity_ruler into nlp_ner pipeline
#entity ruler's patterns(entities) evaluate only if they are not annotaded in training data, so the ”entity_ruler” will only add new entities that match to the patterns only if they don’t overlap with existing entities predicted by the statistical model
#QUEST PWS THA EPISTREFOUME TO LINK AN EINAI APO TRAINING ?PROS TO PARON TO EPISTREFOUME APO TO ENTITY RULER mono se auta pou den exoun oristei apo training loipon
def entity_ruler(nlp_model,model):
    #nlp_ner_corpus2.remove_pipe("entity_ruler")
    if "entity_ruler" not in nlp_model.pipe_names:
        ruler=nlp_model.add_pipe("entity_ruler")
    else:
        ruler=nlp_model.get_pipe("entity_ruler")

    #GIA PATTERNS
    def dict_func(df,linkdf,label):
        patterns=[]
        j=0
        for i in df:
            dict={"label": label}
            dict["pattern"]=[{"LOWER" : str.lower(i)}]
            dict["id"]=linkdf[[1]][1][j]
            patterns.append(dict)
            tokens=tokenizer(i)
            if len(tokens) == 2:
                dict={"label": label}
                dict["pattern"]=[{"LOWER" : str.lower(str(tokens[0]))}, {"IS_PUNCT": True}, {"LOWER" : str.lower(str(tokens[1]))}]
                dict["id"]=linkdf[[1]][1][j]
                patterns.append(dict)
            j=j+1
        ruler.add_patterns(patterns)

    dict_func(df_distr_descr,df,"Distribution_descriptor")
    dict_func(df_life_stages,df1,"Life_stage")
    dict_func(df_body_size,df2,"Body_size")
    #dict_func(df_biol_descr,"Biological Descriptor")

    #in order to put entity ruler into the trained model pipeline
    nlp_model.to_disk(model)

nlp_roberta=spacy.load("model-roberta_dev")
entity_ruler(nlp_roberta,"model-roberta_dev_ruler")



In [39]:

!python -m spacy evaluate model-best_fulldev-eval/ train_dev_full.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   27.08 
NER R   18.40 
NER F   21.91 
SPEED   4928  


=============================== NER (per type) ===============================

                              P       R       F
BODY_SIZE                  9.62    6.10    7.46
DISTRIBUTION_DESCRIPTOR   38.64   26.56   31.48
SAMPLING_DEVICE            0.00    0.00    0.00
LIFE_STAGE                 0.00    0.00    0.00



In [ ]:
nlp_ner_corpusIOB_dev=spacy.load("model-corpusIOB_dev")
nlp_ner_corpusIOB_dev.pipe_names


nlp_ner_corpusIOB_dev_ruler=spacy.load("model-corpusIOB_dev_ruler")
nlp_ner_corpusIOB_dev_ruler.pipe_names

paper2=nlp_ner_corpusIOB_dev("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")
doc=nlp_ner_corpusIOB_dev_ruler("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")

print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in doc.ents])
print("\n")
print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper2.ents])

displacy.render(doc, style="ent" )
displacy.render(paper2, style="ent" )
nlp_ner_corpusIOB_dev_ruler.pipe_names

In [ ]:
from spacy import displacy

combined_model=spacy.load("combined_IOBspacy_roberta_dev_ruler")
combined_model.to_disk("combined_IOBspacy_roberta_dev_ruler")


In [ ]:
#combining 2 ner from trained models (roberta and spacy model)

# nlp_roberta=spacy.load("model-roberta_dev_ruler")
# nlp_roberta.rename_pipe("ner", "ner_roberta")
# nlp_roberta.to_disk("model-roberta_dev_ruler")

nlp1 = spacy.load("model-corpusIOB_dev_ruler")

# Load the second model
nlp2 = spacy.load("model-roberta_dev_ruler")
# Add the components from the second model to the first
for name, component in nlp2.pipeline:
    if name  not in nlp1.pipe_names:
        nlp1.add_pipe(name=name,source=nlp2,factory_name=name)
    
nlp1.to_disk("combined_IOBspacy_roberta_dev_ruler")

nlp1=spacy.load("combined_IOBspacy_roberta_dev_ruler")
doc = nlp1("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")

displacy.render(doc, style="ent" )